In [1]:
import pandas as pds

# Download GTFS

In [4]:
!wget https://www.rtd-denver.com/files/gtfs/google_transit.zip 
!unzip google_transit.zip

Archive:  google_transit.zip
  inflating: agency.txt              
  inflating: areas.txt               
  inflating: calendar.txt            
 extracting: calendar_dates.txt      
  inflating: fare_leg_rules.txt      
  inflating: fare_media.txt          
  inflating: fare_products.txt       
  inflating: fare_transfer_rules.txt  
  inflating: feed_info.txt           
  inflating: networks.txt            
  inflating: routes.txt              
  inflating: route_networks.txt      
  inflating: shapes.txt              
  inflating: stops.txt               
  inflating: stop_areas.txt          
  inflating: stop_times.txt          
  inflating: trips.txt               


In [12]:
pds.read_csv("routes.txt")
pds.read_csv("./shapes.txt")

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,1325342,39.655623,-104.999767,1,NaN
1,1325342,39.656568,-104.999372,2,NaN
2,1325342,39.656568,-104.999372,3,NaN
3,1325342,39.656659,-104.999323,4,NaN
4,1325342,39.656659,-104.999323,5,NaN
...,...,...,...,...,...
712351,1328318,39.611213,-105.016002,234,NaN
712352,1328318,39.611213,-105.016002,235,NaN
712353,1328318,39.611240,-105.016061,236,NaN
712354,1328318,39.611240,-105.016061,237,NaN
